In [142]:
from random import choice, random

class Agent():
    both_cooperate = 1
    both_cheat = -2
    one_cheats = (5, -2)
    num_agents = 0
    mistake_rate = .20
    
    def __init__(self, val=0):
        self.val = val
        self.rounds = 0
        self.history = []
        Agent.num_agents += 1
        self.agent_id = Agent.num_agents
        self.name = 'Agent'
    
    @property
    def score(self):
        return self.val
    @property
    def rounds_played(self):
        return self.rounds
    @property
    def view_history(self):
        return self.history
    
    @classmethod
    def set_mistake_rate(cls, rate):
        cls.mistake_rate = rate    
    @classmethod
    def set_cooperate_amount(cls, amount):
        cls.both_cooperate = amount
    @classmethod
    def set_both_cheat_amount(cls, amount):
        cls.both_cheat = amount
#     @classmethod
#     def set_a_cheat_amount(cls, amount):
#         amount = sorted(amount)
#         cls.a_cheats = (amount[1], amount[0])
#         cls.b_cheats = (amount[0], amount[1])
    @classmethod
    def set_rules(cls, coop, cheat, bothcheat):
        cls.both_cooperate = coop
        cls.both_cheat = bothcheat
        cls.a_cheats = (cheat[0], cheat[1])
        cls.b_cheats = (cheat[1], cheat[0])
        
    def play(self, other):
        self.rounds += 1
        other.rounds += 1
        uncertainty_self = random()
        uncertainty_other = random()
        self_action = self.action(other)
        other_action = other.action(self)
        if uncertainty_self < Agent.mistake_rate:
            if self_action == 'cheat':
                self_action = 'cooperate'
            elif self_action == 'cooperate':
                self_action = 'cheat'
        if uncertainty_other < Agent.mistake_rate:
            if other_action == 'cheat':
                other_action = 'cooperate'
            elif other_action == 'cooperate':
                other_action = 'cheat'        
        
        self.history.append({'opponent_id': other.agent_id, 'opponent_name': other.name,
                             'opponent_action': other_action, 'action': self_action})
        other.history.append({'opponent_id': self.agent_id, 'opponent_name': self.name,
                              'opponent_action': self_action, 'action': other_action})
        
        if self_action == 'cooperate' and other_action == 'cooperate':
            self.val += Agent.both_cooperate
            other.val += Agent.both_cooperate
        elif self_action == 'cheat' and other_action == 'cooperate':
            self.val += Agent.one_cheats[0]
            other.val += Agent.one_cheats[1]
        elif self_action == 'cooperate' and other_action == 'cheat':
            self.val += Agent.one_cheats[1]
            other.val += Agent.one_cheats[0]
        elif self_action == 'cheat' and other_action == 'cheat':
            self.val += Agent.both_cheat
            other.val += Agent.both_cheat

In [289]:
import pandas as pd
from pandas import DataFrame

class Cooperator(Agent):
    def __init__(self):
        super().__init__()
        self.name = 'Cooperator'
        
    def action(self, opponent):
        act = 'cooperate'
        return act
    
    def __repr__(self):
        return 'Cooperator()'
    
class Cheater(Agent):
    def __init__(self):
        super().__init__()
        self.name = 'Cheater'
        
    def action(self, opponent):
        act = 'cheat'
        return act
    
    def __repr__(self):
        return 'Cheater()'
    
class ChaosMonkey(Agent):
    def __init__(self):
        super().__init__()
        self.name = 'ChaosMonkey'
        
    def action(self, opponent):
        act = choice(['cheat', 'cooperate'])
        return act
    
    def __repr__(self):
        return 'Random()'
    
class Tit_for_tat(Agent):
    def __init__(self):
        super().__init__()
        self.name = 'Tit for tat'
        
    def action(self, opponent):
        # may no longer need try/except block
        try:
            for i in opponent.history[::-1]:
                if i['opponent_id'] == self.agent_id:
                    if i['action'] == 'cheat':
                        return 'cheat'
            return 'cooperate'
        except:
            print('error, need to keep this try/except block')
            return 'cooperate'
    
    def __repr__(self):
        return 'Tit_for_tat()'
    
class Suspicious_tit_for_tat(Agent):
    def __init__(self):
        super().__init__()
        self.name = 'Suspicious tit for tat'
        
    def action(self, opponent):
        faced_opponent = False
        for i in opponent.history:
                if i['opponent_id'] == self.agent_id:
                    faced_opponent = True
                    break
        if faced_opponent == False:
            return 'cheat'
        # may no longer need try/except block
        try:
            for i in opponent.history[::-1]:
                if i['opponent_id'] == self.agent_id:
                    if i['action'] == 'cheat':
                        return 'cheat'
            return 'cooperate'
        except:
            print('error, need to keep this try/except block for stft')
            return 'cooperate'
    
    def __repr__(self):
        return 'Suspicious_tit_for_tat()'
    
class Tit_for_two_tats(Agent):
    '''Cheats if her opponent has cheated on her twice in a row.
    (Fool me once, shame on you.)'''
    def __init__(self, name='Tit for two tats'):
        super().__init__()
        self.name = name # NEED TO UPDATE ALL AGENTS WITH CUSTOM NAMES
        
    def action(self, opponent):
        try:
            # convert history to a dataframe so we can filter and count previous actions
            opponent_history = DataFrame(opponent.history)
            opponent_history = opponent_history[opponent_history['opponent_id'] == self.agent_id]
            if (opponent_history.tail(2)['action'] == 'cheat').sum() >= 2:
                return 'cheat'
            return 'cooperate'
        except:
            return 'cooperate'
    
    def __repr__(self):
        return 'Tit_for_two_tats()'

    
class Grudger(Agent):
    '''Never cooperates again with an opponent who has cheated him'''
    def __init__(self, name='Grudger'):
        super().__init__()
        self.name = name # NEED TO UPDATE ALL AGENTS WITH CUSTOM NAMES
        
    def action(self, opponent):
        try:
            # convert history to a dataframe so we can filter and count previous actions
            opponent_history = DataFrame(opponent.history)
            opponent_history = opponent_history[opponent_history['opponent_id'] == self.agent_id]
            if (opponent_history['action'] == 'cheat').sum() >= 0:
                return 'cheat'
            return 'cooperate'
        except:
            return 'cooperate'
    
    def __repr__(self):
        return 'Grudger()'
    
class Generous_tit_for_tat(Agent):
    def __init__(self, forgiveness_rate=.10):
        super().__init__()
        self.name = 'Generous tit for tat'
        self.forgiveness_rate = forgiveness_rate
        
    def action(self, opponent):
        # may no longer need try/except block
        try:
            for i in opponent.history[::-1]:
                if i['opponent_id'] == self.agent_id:
                    if i['action'] == 'cheat':
                        if random() < self.forgiveness_rate:
                            return 'cooperate'
                        else:
                            return 'cheat'
            return 'cooperate'
        except:
            print('error, need to keep this try/except block in gtft')
            return 'cooperate'
    
    def __repr__(self):
        return 'Generous_tit_for_tat()'
    
class Gossip(Agent):
    '''refuses to cooperate with someone who has recently been cheating'''
    def __init__(self, memory=3, forgiveness=0, name='Gossip'):
        super().__init__()
        self.name = name # NEED TO UPDATE ALL AGENTS WITH CUSTOM NAMES
        self.memory = memory
        self.forgiveness = forgiveness
        
    def action(self, opponent):
        try:
            # convert history to a dataframe so we can filter and count previous actions
            opponent_history = DataFrame(opponent.history)
            if (opponent_history.tail(self.memory)['action'] == 'cheat').sum() > self.forgiveness:
                return 'cheat'
            return 'cooperate'
        except:
            return 'cooperate'
        
    def __repr__(self):
        return 'Gossip(memory={}, forgiveness={})'.format(self.memory, self.forgiveness)


class ScrupulousThief(Agent):
    '''Cheats when he doesn't hurt his opponent too much, otherwise tit-for-tat'''
    def __init__(self):
        super().__init__()
        self.name = 'ScrupulousThief'
        
    def action(self, opponent):
        # may no longer need try/except block
        try:
            if Agent.one_cheats[1] > -2:
                return 'cheat'
            for i in opponent.history[::-1]:
                if i['opponent_id'] == self.agent_id:
                    if i['action'] == 'cheat':
                        return 'cheat'
            return 'cooperate'
        except:
            print('error, need to keep this try/except block in ScrupulousThief')
            return 'cooperate'
    
    def __repr__(self):
        return 'ScrupulousThief()'

    
class Opportunist(Agent):
    '''Cheats when it's worth it, otherwise tit-for-tat'''
    def __init__(self, temptation=2, name='Opportunist'):
        super().__init__()
        self.name = name
        self.temptation = temptation
        
    def action(self, opponent):
        # may no longer need try/except block
        try:
            if Agent.one_cheats[0] > self.temptation:
                return 'cheat'
            for i in opponent.history[::-1]:
                if i['opponent_id'] == self.agent_id:
                    if i['action'] == 'cheat':
                        return 'cheat'
            return 'cooperate'
        except:
            print('error, need to keep this try/except block in Opportunist')
            return 'cooperate'
    
    def __repr__(self):
        return 'Opportunist()'

# Lots of random testing blocks below

In [136]:
g = Cooperator()
k = Cooperator()
l = Cheater()

In [139]:
l = Cheater()
m = Tit_for_tat()

for i in range(10):
    l.play(m)
m.score, l.score, m.history

(-10,
 4,
 [{'action': 'cooperate',
   'opponent_action': 'cheat',
   'opponent_id': 8,
   'opponent_name': 'Cheater'},
  {'action': 'cheat',
   'opponent_action': 'cheat',
   'opponent_id': 8,
   'opponent_name': 'Cheater'},
  {'action': 'cheat',
   'opponent_action': 'cheat',
   'opponent_id': 8,
   'opponent_name': 'Cheater'},
  {'action': 'cooperate',
   'opponent_action': 'cooperate',
   'opponent_id': 8,
   'opponent_name': 'Cheater'},
  {'action': 'cooperate',
   'opponent_action': 'cheat',
   'opponent_id': 8,
   'opponent_name': 'Cheater'},
  {'action': 'cheat',
   'opponent_action': 'cheat',
   'opponent_id': 8,
   'opponent_name': 'Cheater'},
  {'action': 'cheat',
   'opponent_action': 'cooperate',
   'opponent_id': 8,
   'opponent_name': 'Cheater'},
  {'action': 'cheat',
   'opponent_action': 'cheat',
   'opponent_id': 8,
   'opponent_name': 'Cheater'},
  {'action': 'cheat',
   'opponent_action': 'cheat',
   'opponent_id': 8,
   'opponent_name': 'Cheater'},
  {'action': 'co

In [106]:
g = Cooperator()
l = Cheater()
for i in range(3):
    l.play(g)
g.score, l.score

(-3, 4)

In [107]:
g = Cooperator()
l = ChaosMonkey()
for i in range(3):
    l.play(g)
g.score, l.score, l.history

(-3,
 11,
 [{'action': 'cheat',
   'opponent_action': 'cooperate',
   'opponent_id': 8,
   'opponent_name': 'Cooperator'},
  {'action': 'cooperate',
   'opponent_action': 'cooperate',
   'opponent_id': 8,
   'opponent_name': 'Cooperator'},
  {'action': 'cheat',
   'opponent_action': 'cooperate',
   'opponent_id': 8,
   'opponent_name': 'Cooperator'}])

In [314]:
def str_to_agent(s):
    agents = {
        'Cooperator()': Cooperator(),
        'Cheater()': Cheater(),
        'Tit_for_tat()': Tit_for_tat(),
        'Generous_tit_for_tat()': Generous_tit_for_tat(),
        'Gossip(memory=3, forgiveness=0)': Gossip(memory=3, forgiveness=0),
        'Gossip(memory=5, forgiveness=1)': Gossip(memory=5, forgiveness=1),
        'Gossip(memory=5, forgiveness=3)': Gossip(memory=5, forgiveness=1),
        'Suspicious_tit_for_tat()': Suspicious_tit_for_tat(),
        'Tit_for_two_tats()': Tit_for_two_tats(),
        'Generous_tit_for_tat()': Generous_tit_for_tat(),
        'ScrupulousThief()': ScrupulousThief(),
        'Opportunist()': Opportunist(),
        'Grudger()': Grudger(),
    }

    return agents[s]

In [326]:
# Tournament rules
Agent.set_rules(.1, (5,-4), -1)
Agent.set_mistake_rate(.25)
evolution_speed = 1
encounters = 2*(len(players)**2)

# Spawn players
players = []
for i in range(2):
    players.append(Cooperator())
for i in range(4):
    players.append(Cheater())
for i in range(2):
    players.append(Tit_for_tat())
for i in range(2):
     players.append(Gossip())
for i in range(2):
     players.append(Gossip(memory=5, forgiveness=3))
for i in range(2):
     players.append(Suspicious_tit_for_tat())        
for i in range(2):
     players.append(Tit_for_two_tats())  
for i in range(2):
     players.append(Generous_tit_for_tat())  
for i in range(2):
     players.append(Grudger())  


for tournament_round in range(100):
    for encounter in range(encounters):
        p1 = choice(players)
        p2 = choice(players)
        if p1.agent_id != p2.agent_id:
            p1.play(p2)
#     for encounter in range(choice([1,2])): #randomize how many times each pair meets
#         for p1 in players:
#             for p2 in players:
#                 if p1.agent_id != p2.agent_id:
#                     p1.play(p2)
    
    # Calculate scores
    standings = {}
    for p in players:
        standings[p.agent_id] = (p, p.name, p.score)
    standings = DataFrame(standings).T.sort_values(2, ascending=False)

    # Evolve population
    for p in standings.head(evolution_speed)[0]:
        players.append(str_to_agent(str(p))) #inefficient, but it works

    for p in standings.tail(evolution_speed).index:
        # This is slow -- O(n) -- and needs to be improved to O(1)
        for pp in players:
            if pp.agent_id == p:
                players.remove(pp)

    # Reset scores
    for p in players:
        p.val = 0
        
standings.drop(0,axis=1)

,1,2
85323,Grudger,48.1
84777,Grudger,18.1
85375,Grudger,17.1
85427,Grudger,13.1
85258,Grudger,12.1
85414,Grudger,11.5
85440,Grudger,6.3
85154,Grudger,2.1
85466,Grudger,0.3
84985,Grudger,0.1


In [303]:
# Single lifetime

# Tournament rules
Agent.set_rules(1, (2,-1), -4)
Agent.set_mistake_rate(.05)
evolution_speed = 1

# Spawn players
players = []
for i in range(4):
    players.append(Cooperator())
for i in range(4):
    players.append(Cheater())
for i in range(2):
    players.append(Tit_for_tat())
for i in range(2):
     players.append(Gossip())
for i in range(2):
     players.append(Gossip(memory=5, forgiveness=3))
for i in range(2):
     players.append(Suspicious_tit_for_tat())        
for i in range(2):
     players.append(Tit_for_two_tats())  
for i in range(2):
     players.append(Generous_tit_for_tat())  
for i in range(2):
     players.append(ScrupulousThief())  
for i in range(2):
     players.append(Opportunist())  

# Random encounters
for i in range(1000):
    p1 = choice(players)
    p2 = choice(players)
    if p1.agent_id != p2.agent_id:
        p1.play(p2)
        
    # People move
    if random() < .1:
        players.remove(choice(players))
        players.append(choice([Cooperator(), Cooperator(), Cheater(), Cheater(), Tit_for_tat(),
                             Gossip(), Gossip(memory=5, forgiveness=3), Suspicious_tit_for_tat(),
                             Tit_for_two_tats(), Generous_tit_for_tat(), ScrupulousThief(),
                             Opportunist()]))
    
    # Sometimes the rewards and penalties change
    if random() < .03:
        Agent.set_rules(choice([0,1,2,3]),
                        (choice([1,2,3,4,5]), choice([0,-1,-2,-3,-4,-5])),
                        choice([0, -1, -2]))
                
# Calculate scores
standings = {}
for p in players:
    standings[p.agent_id] = (p, p.name, p.score)
standings = DataFrame(standings).T.sort_values(2, ascending=False)
        
standings.drop(0,axis=1)

,1,2
69475,Cheater,98
69832,Cheater,85
70204,Cheater,42
70065,Suspicious tit for tat,28
70360,Cheater,25
69887,Generous tit for tat,21
70044,ScrupulousThief,21
70162,Tit for two tats,21
70301,Cheater,16
69818,Cooperator,11


# Dating sim

It might be fun to simulate dating and relationships, because we can establish very different rules and behaviors. Finding someone is good. Finding the *wrong* person is really bad, and no one really benefits.

The most common behavior for most agents is `None`, which means that point totals don't get updated.

In [308]:
class Dater(Agent):
    '''Says yes about 5% of the time'''
    def __init__(self):
        super().__init__()
        self.name = 'Dater'
        
    def action(self, opponent):
        if random() < .05:
            return 'cooperate'
        else:
            return None
    
    def __repr__(self):
        return 'Dater()'
    
class Desperate(Agent):
    '''Usually says yes'''
    def __init__(self):
        super().__init__()
        self.name = 'Desperate'
        
    def action(self, opponent):
        if random() < .90:
            return 'cooperate'
        else:
            return None
    
    def __repr__(self):
        return 'Desperate()'
    
class BadNews(Agent):
    '''Cheats on partners and leaves destruction in their wake'''
    def __init__(self, name='BadNews'):
        super().__init__()
        self.name = name
        
    def action(self, opponent):
        if random() < .90:
            return 'cheat'
        else:
            if random() < .66:
                return None
            return 'cooperate'
    
    def __repr__(self):
        return 'BadNews()'
    
class GossipyDater(Agent):
    '''Stays away from cheaters'''
    def __init__(self, memory=3, choosiness=.05, forgiveness=0, name='GossipyDater'):
        super().__init__()
        self.name = name # NEED TO UPDATE ALL AGENTS WITH CUSTOM NAMES
        self.memory = memory
        self.forgiveness = forgiveness
        self.choosiness = choosiness
        
    def action(self, opponent):
        try:
            # convert history to a dataframe so we can filter and count previous actions
            opponent_history = DataFrame(opponent.history)
            if (opponent_history.tail(self.memory)['action'] == 'cheat').sum() > self.forgiveness:
                return None
            else:
                if random() < self.choosiness:
                    return 'cooperate'
                else:
                    return None
        except:
            return None
        
    def __repr__(self):
        return 'GossipyDater(memory={}, forgiveness={})'.format(self.memory, self.forgiveness)

In [309]:
def str_to_agent(s):
    agents = {
        'Cooperator()': Cooperator(),
        'Cheater()': Cheater(),
        'Tit_for_tat()': Tit_for_tat(),
        'Generous_tit_for_tat()': Generous_tit_for_tat(),
        'Gossip(memory=3, forgiveness=0)': Gossip(memory=3, forgiveness=0),
        'Gossip(memory=5, forgiveness=1)': Gossip(memory=5, forgiveness=1),
        'Gossip(memory=5, forgiveness=3)': Gossip(memory=5, forgiveness=1),
        'Suspicious_tit_for_tat()': Suspicious_tit_for_tat(),
        'Tit_for_two_tats()': Tit_for_two_tats(),
        'Dater()': Dater(),
        'Desperate()': Desperate(),
        'BadNews()': BadNews(),
        'GossipyDater(memory=3, forgiveness=0)': GossipyDater(),
    }

    return agents[s]

In [310]:
# Tournament rules
Agent.set_rules(20, (0,-10), -1)
Agent.set_mistake_rate(.01)
evolution_speed = 1

# Spawn players
players = []
for i in range(2):
    players.append(Dater())
for i in range(1):
    players.append(BadNews())
for i in range(1):
    players.append(Desperate())
for i in range(1):
    players.append(GossipyDater())
    
for tournament_round in range(100):
    for encounter in range(choice([1,2,3])): #randomize how many times each pair meets
        for p1 in players:
            for p2 in players:
                if p1.agent_id != p2.agent_id:
                    p1.play(p2)
    
    # Calculate scores
    standings = {}
    for p in players:
        standings[p.agent_id] = (p, p.name, p.score)
    standings = DataFrame(standings).T.sort_values(2, ascending=False)

    # Evolve population
    for p in standings.head(evolution_speed)[0]:
        players.append(str_to_agent(str(p))) #inefficient, but it works

    for p in standings.tail(evolution_speed).index:
        # This is slow -- O(n) -- and needs to be improved to O(1)
        for pp in players:
            if pp.agent_id == p:
                players.remove(pp)

    # Reset scores
    for p in players:
        p.val = 0
        
standings.drop(0,axis=1)

,1,2
71739,GossipyDater,0
71791,GossipyDater,0
72155,GossipyDater,0
72558,GossipyDater,0
73026,GossipyDater,0


In [187]:
DataFrame(players[0].history)

,action,opponent_action,opponent_id,opponent_name
0,None,None,16612,Dater
1,None,cooperate,16613,Dater
2,None,cooperate,16614,Dater
3,None,None,16615,Dater
4,None,cheat,16616,BadNews
5,None,cheat,16617,BadNews
6,None,cooperate,16618,Desperate
7,None,cooperate,16619,Desperate
8,None,None,16612,Dater
9,None,None,16613,Dater


# TODO:
- ???